In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/home/mrzaizai2k/code_Bao/LLM-with-RAG/notebook


In [2]:
import pandas as pd
from datasets import load_dataset, ClassLabel

from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset

In [3]:
dataset_name = "chibao24/gpt_routing"
# validation_split = 25
datasets = load_dataset(dataset_name, split="train", cache_dir=True)

In [4]:
# Assuming the loaded dataset is named "datasets"
new_features = datasets.features.copy()  # Get dataset features
new_features["label"] = ClassLabel(num_classes = len(datasets.unique("label")), names=datasets.unique("label"))  # Extract unique labels and cast
datasets = datasets.cast(new_features)
ddict  = datasets.train_test_split(test_size=0.25, stratify_by_column="label", shuffle =True)
train_ds, val_ds = ddict["train"], ddict["test"]
ddict_val_test  = val_ds.train_test_split(test_size=0.5, stratify_by_column="label", shuffle =True)
val_ds, test_ds = ddict_val_test["train"], ddict_val_test["test"]

In [5]:
train_ds

Dataset({
    features: ['ID', 'text', 'Language', 'Class_text', 'label'],
    num_rows: 21
})

In [6]:
val_ds

Dataset({
    features: ['ID', 'text', 'Language', 'Class_text', 'label'],
    num_rows: 4
})

In [7]:
test_ds

Dataset({
    features: ['ID', 'text', 'Language', 'Class_text', 'label'],
    num_rows: 4
})

In [8]:
# Load a SetFit model from Hub
sentence_model = "sentence-transformers/all-MiniLM-L6-v2"
model = SetFitModel.from_pretrained(
    sentence_model,
    labels=[0, 1],
)

/home/mrzaizai2k/code_Bao/LLM-with-RAG/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [9]:

args = TrainingArguments(
    batch_size=4,
    num_epochs=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

args.eval_strategy = args.evaluation_strategy

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    metric="accuracy",
    column_mapping={"text": "text", "label": "label"}  # Map dataset columns to text/label expected by trainer
)

trainer.train()


Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/21 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 242
  Batch size = 4
  Num epochs = 4
  Total optimization steps = 244


Epoch,Training Loss,Validation Loss,Embedding Loss,Rate
1,No log,No log,0.083800,0.000017
2,No log,No log,0.057200,0.000011
3,No log,No log,0.052300,0.000006
4,No log,No log,0.054100,0.000000


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Loading best SentenceTransformer model from step 183.


In [10]:
metrics = trainer.evaluate(test_ds)
print(metrics)

Applying column mapping to the evaluation dataset
***** Running evaluation *****


{'accuracy': 0.25}


In [11]:
trainer.push_to_hub("chibao24/model_routing_few_shot")

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/chibao24/model_routing_few_shot/commit/f7acc7ee67cfe69dc930300ad9dec2d8e4ba0bcd', commit_message='Add SetFit model', commit_description='', oid='f7acc7ee67cfe69dc930300ad9dec2d8e4ba0bcd', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
model=SetFitModel.from_pretrained("chibao24/model_routing_few_shot")

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config_setfit.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

model_head.pkl:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

In [16]:
preds=model.predict(["what is the capital of VietNam", "explain in detail the karger min cut and it's complexity"])
preds

[0, 1]